In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from pandas import read_csv
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# from xgboost import XGBRegressor

%matplotlib inline

In [12]:
SB_df_orig = pd.read_csv("../input/sberbank_train.csv", parse_dates=['timestamp'])
SB_df_macro_orig = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])

SB_df = pd.read_csv("../input/sberbank_train.csv", parse_dates=['timestamp'])
SB_df_macro = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])

# add new collumns
SB_df['year_of_sale'] = SB_df['timestamp'].apply(lambda x: x.strftime("%Y"))
SB_df_macro['year_of_sale'] = SB_df_macro['timestamp'].apply(lambda x: x.strftime("%Y"))

SB_df['month_of_sale_num'] = SB_df['timestamp'].apply(lambda x: x.strftime("%m"))
SB_df_macro['month_of_sale_num'] = SB_df_macro['timestamp'].apply(lambda x: x.strftime("%m"))

SB_df['Month_Year_of_sale_num'] = SB_df['year_of_sale'].astype(int)*100 + SB_df['month_of_sale_num'].astype(int)
SB_df_macro['Month_Year_of_sale_num'] =  SB_df_macro['year_of_sale'].astype(int)*100 + SB_df_macro['month_of_sale_num'].astype(int)

SB_df_numeric = SB_df.select_dtypes(include=['float64', 'int64'])
SB_df_macro_numeric = SB_df_macro.select_dtypes(include=['float64', 'int64'])

In [13]:
SB_df_numeric_not_null = SB_df_numeric.fillna(0)
SB_df_macro_numeric_not_null = SB_df_macro_numeric.fillna(0)

# SB_joined_df = pd.merge(SB_df_numeric_not_null, SB_df_macro_numeric_not_null, left_on='Month_Year_of_sale_num', right_on='Month_Year_of_sale_num')

In [14]:
# max_depth=20, 
regr = RandomForestRegressor(max_depth=30, random_state=0, n_estimators  = 500)

X_des = SB_df_numeric_not_null.astype(np.float64)
y_des = X_des['price_doc']

Xy_train, Xy_test = train_test_split(X_des, train_size=0.7, 
                            random_state=7)

X_des = X_des.drop('price_doc', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_des,y_des, train_size=0.7, 
                            random_state=7)

In [ ]:
# (type(set(X_train.columns)))

In [15]:
regr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [16]:
Xy_train['pred_price_doc'] = regr.predict(X_train).astype(long)
Xy_test['pred_price_doc'] = regr.predict(X_test).astype(long)

In [17]:
all_rmse = np.sqrt(MSE(Xy_train['price_doc'],
                       Xy_train['pred_price_doc']))
print "RMSE = {:.2f}".format(all_rmse)

RMSE = 1036725.87


In [18]:
all_rmse = np.sqrt(MSE(Xy_test['price_doc'],
                       Xy_test['pred_price_doc']))
print "RMSE = {:.2f}".format(all_rmse)

RMSE = 2545517.61


In [19]:
# start loading real test data

SB_TEST_df = pd.read_csv("../input/sberbank_test.csv", parse_dates=['timestamp'])

# add new collumns
SB_TEST_df['year_of_sale'] = SB_TEST_df['timestamp'].apply(lambda x: x.strftime("%Y"))

SB_TEST_df['month_of_sale_num'] = SB_TEST_df['timestamp'].apply(lambda x: x.strftime("%m"))

SB_TEST_df['Month_Year_of_sale_num'] = SB_TEST_df['year_of_sale'].astype(int)*100 + SB_TEST_df['month_of_sale_num'].astype(int)

SB_TEST_df_numeric = SB_TEST_df.select_dtypes(include=['float64', 'int64'])

SB_TEST_df_numeric_not_null = SB_TEST_df_numeric.fillna(0)

# x = SB_TEST_df_numeric_not_null.copy()

# set(SB_TEST_df_numeric_not_null.columns) - set(X_train.columns)

# X_train['Month_Year_of_sale_num']

SB_TEST_df_numeric_not_null['price_doc'] = regr.predict(SB_TEST_df_numeric_not_null).astype(long)

final_sub = SB_TEST_df_numeric_not_null[['id', 'price_doc']]
# .set_index('id')

final_sub.to_csv('slav_sub.csv', index=False)